# Hendricks Group

# Hidden Ancestry Example Notebook

_In the following notebook, we solve an example Hidden Ancestries problem._ 

* We use simulated SNPs with minor allele frequencies for $K=5$ ancestries -- European ancestries, African ancestries, South Asian ancestries, East Asian ancestries, and Native American ancestires. 

* We numerically solve for the 5 ancestry's true proportions in the _observed_ population, which is the vector $\pi^*:=(\pi_1,\pi_2,\pi_3,\pi_4, \pi_5)$.
    * $\pi_1$ denotes the proportion of European ancestries in the observed population
    * $\pi_2$ denotes the proportion of African ancestries in the observed population
    * $\pi_3$ denotes the proportion of South Asian ancestries in the observed population
    * $\pi_4$ denotes the proportion of East Asian ancestries in the observed population
    * $\pi_5$ denotes the proportion of Native American ancestries in the observed population

* In this notebook, we work with an example data set $D$ with $N=10,000$ SNPs and ensure that our Python script correctly uses the data to solve for these ancestry proportions using Sequential Least Squares Quadratic Programming, or SLSQP.

* The cell below calls the generalized HA script, which is the main feature of the HA Python package. Then we can access the functions inside of the script to solve an example problem.

In [1]:
%run HA_script.py

* In the cell below, we read in the example data set provided with this package, called "packagedata.csv".
* We use Pandas to convert from the CSV format to an array endowed with numerical linear algebra properties Python understands.
* How $D$ is formatted matters a lot -- order matters...
    * $D$ contains (for now) an indexing column containing the natural numbers up to $N$ (I think Pandas might be adding this???), SNP number (location on genome), Chromosome number, the $5$ minor allele frequencies of the $K=5$ ancestires (in this case), and finally, any (gnomAD) observed allele frequencies we wish to model. 
    * We only need certain columns of $D$ to solve our example problem...
        * That is, we only need the minor allele frequencies and whichever observed allele frequency we are modeling, which should be $K+1$ columns of D. In our case, we need 6 of the total 11 columns.
* We print out the first 5 rows of $D$ to take a look at its structure and check for basic correctness in what we _think_ we are working with!

In [2]:
import pandas as pd

# Read in the data
D = pd.read_csv("packagedata.csv")

D.head(5) ### Look at the first 5 rows

,SNP,CHR,ref_eur,ref_afr,ref_sas,ref_eas,ref_nam,gnomad_afr,gnomad_amr,gnomad_oth
0,rs1695824,1,0.210394,0.902781,0.693261,0.990080,0.98837,0.779977,0.615476,0.334254
1,rs7519837,1,0.337873,0.929563,0.712678,0.870030,0.79070,0.830306,0.548463,0.376384
2,rs7545940,1,0.345311,0.375000,0.416164,0.373990,0.27910,0.391494,0.356974,0.381919
3,rs897632,1,0.231473,0.308540,0.325159,0.260915,0.67440,0.292203,0.369668,0.280037
4,rs12084604,1,0.001238,0.162712,0.000000,0.000000,0.00000,0.129906,0.010613,0.011029


* In the cell below, we specify the number of ancestries, $K$, (here we have 5) and optionally choose an initial iterate.
    * Choosing an initial iterate is optional, and so we leave this command commented out.
    * The default initial iterate is $\pi^{(0)}=\frac{1}{K}(1,\ldots,1)\in \mathbb{R}^{K}$.
    * The initial iterate must be a $K \times 1$ (column) or $1 \times K$ (row) vector (the HA script can handle either shape)

In [3]:
K=5 # User must specify number of ancestries!
pi_0 = [[0.3,0.1,0.2,0.1,0.3]] # You do not have to provide the initial iterate, but you may.
np.shape(pi_0)

(1, 5)

* In the cell below, we quickly check that we have specified a data matrix $D$ and total number of ancestries $K$ that match the number of SNPs we think we are working with as well as the correct ancestry number.

In [4]:
print('our problem includes', np.shape(D)[0], 'SNPs, and', K, 'ancestries')

our problem includes 10000 SNPs, and 5 ancestries


* Finally, in the cell below, we apply the HA function to the data $D$, with intial iterate $\pi^{(0)}$ (optional), and the number of ancestries, $K=5$. 
    * We can also provide different observation columns if we choose.
* The HA function will output an array containing the numerical solution, $\pi^{final}$, the number of SLSQP iterations taken to find the numerical solution, and the total runtime (in seconds) of SLSQP.

In [5]:
output_array = HA(D,K) # This line runs the HA function inside the HA_script using default initial iterate and default observed
# output_array = HA(D,K,obs=2) # This line runs the HA function inside the HA_script using default initial iterate and second observed
# output_array = HA(D,K,obs=3) # This line runs the HA function inside the HA_script using default initial iterate and third observed
# output_array = HA(D,K,pi_0) # This line runs the HA function inside the HA_script with a particular choice for initial iterate


print('numerical solution via SLSQP, pi_final = ',output_array[0])
print()
print('number of SLSQP iterations:',output_array[1])
print()
print('runtime:',output_array[2],'seconds')

numerical solution via SLSQP, pi_final =  [0.15612835 0.82430497 0.00691932 0.00316927 0.00947809]

number of SLSQP iterations: 12

runtime: 0.0207805 seconds


### _Above we see a printout of the numerical solution, $\pi^{final}$, the number of SLSQP iterations, and the time in seconds of the run._

In detail...

* The numerical solution we have found is given by $$\pi^\text{final}\approx (0.156,0.824,0.007,0.003,0.009)$$

    * $\pi_1^\text{final}\approx 0.156$ denotes the proportion of European ancestries in the observed population
    * $\pi_2^\text{final}\approx 0.824$ denotes the proportion of African ancestries in the observed population
    * $\pi_3^\text{final}\approx 0.007$ denotes the proportion of South Asian ancestries in the observed population
    * $\pi_4^\text{final}\approx 0.003$ denotes the proportion of East Asian ancestries in the observed population
    * $\pi_5^\text{final}\approx 0.009$ denotes the proportion of Native American ancestries in the observed population
    * Recall that we chose the gnomAD African sample for our observed population in this example.
    
* SLSQP went through about 10 iterations to obtain this numerical solution

* The runtime of the script/computational process should be less than a tenth of a second!
    * I am witnessing run times closer to a hundredth of a second!